In [1]:
import requests
import pandas as pd
import time
from datetime import datetime


In [17]:
WALLET_ADDRESS = "0x742d35Cc6634C0532925a3b844Bc454e4438f44e" 
ETHERSCAN_API_KEY = "HBVI4CYSPKQWC8ZUTTFKTBNPDST6N3IZTW"


In [18]:
def fetch_erc20_transfers(wallet):
    url = "https://api.etherscan.io/v2/api"
    params = {
        "chainid": 1,  # Ethereum mainnet
        "module": "account",
        "action": "tokentx",
        "address": wallet,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": ETHERSCAN_API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "result" not in data:
        print(data)
        return []

    return data["result"]


In [19]:
def fetch_erc20_transfers(wallet):
    url = "https://api.etherscan.io/v2/api"
    params = {
        "chainid": 1,  # Ethereum mainnet
        "module": "account",
        "action": "tokentx",
        "address": wallet,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": ETHERSCAN_API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "result" not in data:
        print(data)
        return []

    return data["result"]


In [20]:
erc20_txs = fetch_erc20_transfers(WALLET_ADDRESS)
len(erc20_txs)


10000

In [21]:
def wallet_age_days_from_erc20(transfers):
    if not transfers:
        return 0

    first_ts = int(transfers[0]["timeStamp"])
    first_date = datetime.utcfromtimestamp(first_ts)
    return (datetime.utcnow() - first_date).days


wallet_age_days = wallet_age_days_from_erc20(erc20_txs)
wallet_age_days


2799

In [22]:
def activity_consistency(transfers):
    if not transfers:
        return 0

    df = pd.DataFrame(transfers)
    df["timeStamp"] = pd.to_datetime(df["timeStamp"].astype(int), unit="s")

    weekly_counts = (
        df
        .set_index("timeStamp")
        .resample("W")
        .size()
    )

    return weekly_counts.std() / weekly_counts.mean() if weekly_counts.mean() > 0 else 0


consistency_score = activity_consistency(erc20_txs)
consistency_score


0.8697955218927963

In [23]:
def token_diversity(transfers):
    tokens = {tx["tokenSymbol"] for tx in transfers if tx.get("tokenSymbol")}
    return len(tokens)


unique_tokens = token_diversity(erc20_txs)
unique_tokens


322

In [37]:
def risk_score(wallet_age_days, consistency, token_count):

    if wallet_age_days < 30 or token_count == 0:
        return 20  # forces HIGH risk

    score = 0

    # Wallet age
    if wallet_age_days > 365:
        score += 40
    elif wallet_age_days > 90:
        score += 25
    else:
        score += 10

    # Consistency (only meaningful if wallet is active)
    if token_count > 5:
        if consistency < 1:
            score += 30
        elif consistency < 2:
            score += 15
        else:
            score += 5
    else:
        score += 0  # no reward for consistency if inactive

    # Token diversity
    if token_count > 10:
        score += 30
    elif token_count > 3:
        score += 15
    else:
        score += 5

    score = min(score, 100)
    return score


In [28]:
score = risk_score(wallet_age_days, consistency_score, unique_tokens)
score


100

In [29]:
def risk_tier(score):
    if score >= 75:
        return "LOW"
    elif score >= 45:
        return "MEDIUM"
    else:
        return "HIGH"


risk = risk_tier(score)
risk


'LOW'

In [30]:
def evaluate_wallet(wallet_address):
    erc20_txs = fetch_erc20_transfers(wallet_address)

    wallet_age = wallet_age_days_from_erc20(erc20_txs)
    consistency = activity_consistency(erc20_txs)
    tokens = token_diversity(erc20_txs)

    score = risk_score(wallet_age, consistency, tokens)
    tier = risk_tier(score)

    return {
        "wallet": wallet_address,
        "wallet_age_days": wallet_age,
        "consistency_score": round(consistency, 3),
        "unique_tokens": tokens,
        "score": score,
        "risk_tier": tier
    }


In [31]:
evaluate_wallet("0x742d35Cc6634C0532925a3b844Bc454e4438f44e")


{'wallet': '0x742d35Cc6634C0532925a3b844Bc454e4438f44e',
 'wallet_age_days': 2800,
 'consistency_score': 0.87,
 'unique_tokens': 322,
 'score': 100,
 'risk_tier': 'LOW'}

In [32]:
evaluate_wallet("0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0")


{'wallet': '0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0',
 'wallet_age_days': 3302,
 'consistency_score': 11.947,
 'unique_tokens': 1027,
 'score': 75,
 'risk_tier': 'LOW'}

In [38]:
evaluate_wallet("0x7C706A29Daf45A942D740cB24a63559376c01d1B")


{'wallet': '0x7C706A29Daf45A942D740cB24a63559376c01d1B',
 'wallet_age_days': 0,
 'consistency_score': 0,
 'unique_tokens': 0,
 'score': 20,
 'risk_tier': 'HIGH'}